In [3]:
# imports
%matplotlib inline
from pathlib import Path
from PIL import Image, TiffTags
import matplotlib.pyplot as plt

In [4]:
root_dir_path = Path('/Users/dlisla/shared_with_docker/')
collection_dir_path = root_dir_path.joinpath('agrtfhs')

to_process_dir_path = collection_dir_path.joinpath('3.toQC')

tif_paths_list = sorted(to_process_dir_path.glob('**/*.tif'))
number_of_tifs = len(tif_paths_list)
print(f'number_of_tifs = {number_of_tifs}')

number_of_tifs = 3244


In [28]:
# which TIFF fields do we need to check for bit depth?
image = Image.open(tif_paths_list[0])
for tag in image.tag_v2:
    print(tag)
    print(TiffTags.lookup(tag))

256
TagInfo(value=256, name='ImageWidth', type=4, length=1, enum={})
257
TagInfo(value=257, name='ImageLength', type=4, length=1, enum={})
258
TagInfo(value=258, name='BitsPerSample', type=3, length=0, enum={})
259
TagInfo(value=259, name='Compression', type=3, length=1, enum={'Uncompressed': 1, 'CCITT 1d': 2, 'Group 3 Fax': 3, 'Group 4 Fax': 4, 'LZW': 5, 'JPEG': 6, 'PackBits': 32773})
262
TagInfo(value=262, name='PhotometricInterpretation', type=3, length=1, enum={'WhiteIsZero': 0, 'BlackIsZero': 1, 'RGB': 2, 'RGB Palette': 3, 'Transparency Mask': 4, 'CMYK': 5, 'YCbCr': 6, 'CieLAB': 8, 'CFA': 32803, 'LinearRaw': 32892})
273
TagInfo(value=273, name='StripOffsets', type=4, length=0, enum={})
339
TagInfo(value=339, name='SampleFormat', type=3, length=0, enum={})
277
TagInfo(value=277, name='SamplesPerPixel', type=3, length=1, enum={})
279
TagInfo(value=279, name='StripByteCounts', type=4, length=0, enum={})
282
TagInfo(value=282, name='XResolution', type=5, length=1, enum={})
283
TagInfo

In [29]:
# let's get data for our first image
tag_dict = {'BitsPerSample': 258,
            'Compression': 259,
            'SamplesPerPixel': 277,
           }
for key, value in tag_dict.items():
    print(f'{key} : {image.tag_v2[value]}')

BitsPerSample : (8, 8, 8)
Compression : 1
SamplesPerPixel : 3


In [44]:
# can use Image.getcolors() to identify if the image contains, color, grayscale, or only bitonal data
# if only 1-2 colors returned then bitonal (all black or all white = bitonal, too!)
# if 3-256 colors returned then grayscale
# if 257+ colors, it returns None and it has color data
color_list = image.getcolors()
if color_list is None:
    number_of_colors = 'Over 256 colors'
else:
    number_of_colors = len(color_list)
print(f'Number of colors in the image: {number_of_colors}')

Number of colors in the image: Over 256 colors


In [46]:
bit_depth_lookup_dict = {24: 'rgb',
                         8: 'gray',
                         1: 'bitonal',
                        }
def get_bit_depth_from_colors(image):
    color_list = image.getcolors()
    if color_list is None:  # image is over 256 colors
        bit_depth = 24
    else:
        number_of_colors = len(color_list)
        if number_of_colors > 2:
            bit_depth = 8
        elif number_of_colors > 0:
            bit_depth = 1
        else:
            bit_depth = None
    return bit_depth

In [47]:
# NOTE: writing this as a for-loop, but need to come back and
# refactor into OOP after getting logic and processes down
for image_path in tif_paths_list[:25]:
    image = Image.open(image_path)
    bit_depth = get_bit_depth_from_colors(image)

    print(f'{image_path.name} --> {bit_depth_lookup_dict[bit_depth]}')

000001.tif --> rgb
000002.tif --> rgb
000003.tif --> gray
000004.tif --> bitonal
000005.tif --> gray
000006.tif --> bitonal
000007.tif --> gray
000008.tif --> bitonal
000009.tif --> bitonal
000010.tif --> bitonal
000011.tif --> rgb
000012.tif --> bitonal
000001.tif --> rgb
000002.tif --> rgb
000003.tif --> gray
000004.tif --> bitonal
000005.tif --> gray
000006.tif --> gray
000007.tif --> gray
000008.tif --> gray
000009.tif --> gray
000010.tif --> bitonal
000011.tif --> bitonal
000012.tif --> rgb
000001.tif --> rgb


In [ ]:
# what do we need to check and in what order?
# if continuous tone we should verify that it's not bitonal by getting image as 
check_if_color_path_list = []
grayscale_path_list = []
if image.tag_v2[258] == '(8, 8, 8)': # image is color
    check_if_color_path_list.append(tif_path)
elif image.tag_v2[258] == 8:
    check_if_grayscale

In [ ]:
# open in Preview
image.show()